In [1]:
import math
import tensorflow as tf
import numpy as np
import numba as nb
from numpy.random import rand

In [26]:
def mult(a, b, res):
    n = len(a)
    m = len(b)
    for i in range(n):
        for j in range(m):
            res[i,j] = a[i]*b[j]

class CrossEntropy:
    def __init__(self, num_classes, batch = 1, eps = 1e-18):
        self.num_classes = num_classes
        self.drop_grad = np.zeros(self.num_classes, dtype='float32')
        self.batch = batch
        self.eps = eps
    def calc(self, X, class_labels):
        self.X = X
        if self.batch == 1:
            label = class_labels
            self.labels = [class_labels]
            return -np.log(X[label]+self.eps)
        else:
            self.labels = class_labels
    def grad(self):
        for label in self.labels:
            self.drop_grad[label] = -1.0/(self.X[label]*self.batch)
        return self.drop_grad
    
class SoftMax:
    def __init__(self, n):
        self.n = n
        self.tmp = np.empty((n,n), dtype='float32')
        self.softmax = np.empty(n, dtype='float32')
    def calc(self, X):
        self.softmax = np.exp(X)
        self.softmax /= np.sum(self.softmax)
        return self.softmax
    def grad(self, vec_top_grad):
        mult(-self.softmax, self.softmax, self.tmp)
        self.tmp[np.diag_indices_from(self.tmp)] = self.softmax*(1.0-self.softmax)
        return np.dot(vec_top_grad, self.tmp)
    
def initW(n_input, n_output):
    return (2*rand(n_input*n_output).reshape(n_input,n_output)).astype('float32'), (2*rand(n_output)).astype('float32')

class Net:
    def __init__(self, layers, num_classes):
        self.layers = layers
        self.n_layers = len(layers)
        self.cross = CrossEntropy(num_classes)
    def calc(self, x):
        res = None
        for layer in self.layers:
            res = layer.calc(x)
            x = res
        return res
    def fit(self, X, Y, n):
        for it in range(n):
            for x,y in zip(X,Y):
                res = self.calc(x)
                # cross res
                self.cross.calc(res, y)
                gr = self.cross.grad()
                for layer in reversed(self.layers):
                    gr = layer.grad(gr)

In [27]:
class FullConnected:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.n_input = len(W)
        self.n_output = len(b)
        self.new_grad = np.zeros((self.n_input,self.n_output), dtype='float32')
        self.res = np.zeros(self.n_output, dtype='float32')
    def calc(self, x):
        self.x = x
        #vec(x)*matr(W)=vec(y)        
        np.dot(x, self.W,  out=self.res)
        self.res += self.b
        return self.res
    def grad(self, vec_top_grad):
        # поиск производной выхода y по входу x: d(y)/d(x)
        next_grad = np.dot(self.W, vec_top_grad) #порядок умножения?
        
        # поиск производной по W
        mult(self.x, vec_top_grad, self.new_grad)
        self.W -= 0.5*self.new_grad #0.1 коэффициент градиентного спуска
        
        # поиск производной по b
        self.b -= 0.5*vec_top_grad
        return next_grad

#grad_softmax([1,1,1],[1,-1,0])
num_classes = 4
cross = CrossEntropy(num_classes)

x = np.array([1.,0.5]).astype('float32')
label = 1
np.random.seed(0)
l1 = FullConnected(*initW(2,3))
l2 = FullConnected(*initW(3,num_classes))
print(l1.W)
softmax = SoftMax(num_classes)

res1 = l1.calc(x)
res2 = l2.calc(res1)
res = softmax.calc(res2)
print('res = ',res, '\n\n')
cross.calc(res, label)

gr1 = cross.grad()
print('gr1 =',gr1)
gr2 = softmax.grad(gr1)
print('gr2 =',gr2)
gr3 = l2.grad(gr2)
print('gr3 =',gr3)
gr4 = l1.grad(gr3)
print(gr4)


print('res')
res1 = l1.calc(x)
res2 = l2.calc(res1)
res = softmax.calc(res2)
print(res)
cross.calc(res, label)

[[1.097627  1.4303787 1.2055267]
 [1.0897664 0.8473096 1.2917882]]
res =  [0.9862742  0.00518497 0.00588081 0.00265998] 


gr1 = [   0.      -192.86523    0.         0.     ]
gr2 = [ 0.9862742  -0.99481505  0.00588081  0.00265998]
gr3 = [-0.80964047  1.6855811   0.10957771]
[1.6544349 0.6874413]
res
[5.3978783e-13 9.9999964e-01 2.1849610e-07 1.3972232e-07]


3.576279326012052e-07

In [33]:
num_classes = 4
cross = CrossEntropy(num_classes)

x = np.array([1.,0.5], dtype='float32')
label = 1
np.random.seed(0)
l1 = FullConnected(*initW(2,3))
l2 = FullConnected(*initW(3,num_classes))
softmax = SoftMax(num_classes)

net = Net([l1,l2, softmax],num_classes)
net.fit([x], [label], 100)
print(net.calc(x))

[5.3914978e-13 9.9999964e-01 2.1822309e-07 1.3956264e-07]


In [5]:
np.random.rand(4).reshape(2,2)
A,B = ['x1','x2'],['y1', 'y2']
for a,b in zip(A,B):
    print(a,b)

x1 y1
x2 y2
